In [1]:
import numpy as np
import pandas as pd
import json
from urllib.request import urlopen
import os

# State Income Data

Constructs a data set of real income per capita for the continental United States from 1840 to the present.

Nominal income per capita for 1840, 1880, a 1900 were found in Appendix A in "Interregional Differences in Per Capita Income, Population, and Total Income, 1840-1950" by Richard Easterlin in <ins>Trends in the American Economy in the Nineteenth Century</ins> (https://www.nber.org/books-and-chapters/trends-american-economy-nineteenth-century).

The CPI for 1840, 1880, and 1900 was taken from "<ins>Bicentennial Edition: Historical Statistics of the United States, Colonial Times to 1970</ins> (https://www.census.gov/library/publications/1975/compendia/hist_stats_colonial-1970.html)


Income data from 1929 are obtained from the BEA.

## Preliminaries

In [2]:
# Import BEA API key or set manually to variable api_key
try:
    items = os.getcwd().split('/')[:3]
    items.append('bea_api_key.txt')
    path = '/'.join(items)
    with open(path,'r') as api_key_file:
        api_key = api_key_file.readline()

except:
    api_key = None

In [3]:
# Dictionary of state abbreviations
stateAbbr = {
u'Alabama':u'AL',
u'Alaska *':u'AK',
u'Arizona':u'AZ',
u'Arkansas':u'AR',
u'California':u'CA',
u'Colorado':u'CO',
u'Connecticut':u'CT',
u'Delaware':u'DE',
u'District of Columbia':u'DC',
u'Florida':u'FL',
u'Georgia':u'GA',
u'Hawaii *':u'HI',
u'Idaho':u'ID',
u'Illinois':u'IL',
u'Indiana':u'IN',
u'Iowa':u'IA',
u'Kansas':u'KS',
u'Kentucky':u'KY',
u'Louisiana':u'LA',
u'Maine':u'ME',
u'Maryland':u'MD',
u'Massachusetts':u'MA',
u'Michigan':u'MI',
u'Minnesota':u'MN',
u'Mississippi':u'MS',
u'Missouri':u'MO',
u'Montana':u'MT',
u'Nebraska':u'NE',
u'Nevada':u'NV',
u'New Hampshire':u'NH',
u'New Jersey':u'NJ',
u'New Mexico':u'NM',
u'New York':u'NY',
u'North Carolina':u'NC',
u'North Dakota':u'ND',
u'Ohio':u'OH',
u'Oklahoma':u'OK',
u'Oregon':u'OR',
u'Pennsylvania':u'PA',
u'Rhode Island':u'RI',
u'South Carolina':u'SC',
u'South Dakota':u'SD',
u'Tennessee':u'TN',
u'Texas':u'TX',
u'Utah':u'UT',
u'Vermont':u'VT',
u'Virginia':u'VA',
u'Washington':u'WA',
u'West Virginia':u'WV',
u'Wisconsin':u'WI',
u'Wyoming':u'WY'
}

# List of states in the US
stateList = [s for s in stateAbbr]

## Deflator data

In [4]:
# Obtain data from BEA
gdp_deflator = urlopen('http://apps.bea.gov/api/data/?UserID='+api_key+'&method=GetData&datasetname=NIPA&TableName=T10109&TableID=13&Frequency=A&Year=X&ResultFormat=JSON&')

# Parse result
result = gdp_deflator.read().decode('utf-8')
json_response = json.loads(result)

# Import to DataFrame and organize
df = pd.DataFrame(json_response['BEAAPI']['Results']['Data'])
df['DataValue'] = df['DataValue'].astype(float)
df = df.set_index(['LineDescription',pd.to_datetime(df['TimePeriod'])])
df.index.names = ['line description','Year']

# Extract price level data
data_p = df['DataValue'].loc['Gross domestic product']/100
data_p.name = 'price level'
data_p = data_p.sort_index()
data_p

Year
1929-01-01    0.08778
1930-01-01    0.08457
1931-01-01    0.07587
1932-01-01    0.06700
1933-01-01    0.06514
               ...   
2019-01-01    1.04008
2020-01-01    1.05381
2021-01-01    1.10213
2022-01-01    1.17973
2023-01-01    1.22273
Name: price level, Length: 95, dtype: float64

In [5]:
base_year = json_response['BEAAPI']['Results']['Notes'][0]['NoteText'].split('Index numbers, ')[-1].split('=')[0]

with open('../csv/state_income_metadata.csv','w') as newfile:
    newfile.write(',Values\n'+'base_year,'+base_year)

## Per capita income data

In [6]:
# Obtain data from BEA
state_y_pc = urlopen('http://apps.bea.gov/api/data/?UserID='+api_key+'&method=GetData&DataSetName=Regional&TableName=SAINC1&LineCode=3&Year=ALL&GeoFips=STATE&ResultFormat=JSON')

# Parse result
result = state_y_pc.read().decode('utf-8')
json_response = json.loads(result)

# Import to DataFrame and organize
df = pd.DataFrame(json_response['BEAAPI']['Results']['Data'])
df.GeoName = df.GeoName.replace(stateAbbr)
df = df.set_index(['GeoName',pd.DatetimeIndex(df['TimePeriod'])])
df.index.names = ['State','Year']
df['DataValue'] = df['DataValue'].replace('(NA)',np.nan)


# Extract income data
data_y = df['DataValue'].str.replace(',','').astype(float)
data_y.name = 'income'
data_y = data_y.unstack('State')
data_y = data_y.sort_index()
data_y = data_y.divide(data_p,axis=0)
data_y

State,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,...,TN,TX,UT,United States,VA,VT,WA,WI,WV,WY
Year,,,,,,,,,,,,,,,,,,,,,
1929-01-01,0.000000,3634.085213,3451.811347,6812.485760,11323.763955,7188.425609,11722.488038,15083.162452,11745.272272,5912.508544,...,4249.259512,5399.863295,6242.879927,7963.089542,4955.570745,7177.033493,8430.166325,7632.718159,5228.981545,7655.502392
1930-01-01,0.000000,3109.849829,2648.693390,6125.103465,10512.001892,6787.276812,10961.333806,15537.424619,10145.441646,5533.877261,...,3795.672224,4812.581294,5841.314887,7343.029443,4576.090812,6763.627764,7768.712309,6917.346577,4812.581294,6881.873005
1931-01-01,0.000000,2899.696850,2728.351127,5667.589297,9924.871491,6221.167787,10636.615263,16515.091604,10228.021616,5232.634770,...,3598.260182,4547.251878,4903.123764,6959.272440,4889.943324,6207.987347,7038.355081,6181.626466,4665.875840,6287.069988
1932-01-01,0.000000,2373.134328,2268.656716,4805.970149,8716.417910,5328.358209,9358.208955,16522.388060,8820.895522,4731.343284,...,2895.522388,3910.447761,4537.313433,6000.000000,4268.656716,5417.910448,6000.000000,5402.985075,3835.820896,5597.014925
1933-01-01,0.000000,2517.654283,2348.787228,4743.629107,8443.352779,5419.097329,9072.766349,14507.215229,8688.977587,4436.598096,...,3085.661652,3899.293829,4559.410500,5756.831440,4390.543445,5173.472521,5772.182990,5127.417869,3976.051581,5710.776788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01,58234.943466,41346.819475,42065.033459,45855.126529,61701.022998,58897.392508,71314.706561,80413.045150,51674.871164,51572.955926,...,47005.038074,51195.100377,46300.284593,53406.468733,56118.760095,52166.179525,60961.656796,50346.127221,40984.347358,59479.078532
2020-01-01,58737.343544,43543.902601,44739.564058,49470.967252,66483.521697,61540.505404,73431.643275,85122.555299,52929.845038,53672.863230,...,49276.435031,52303.546180,49108.473064,56132.509655,58334.993974,55017.507900,64218.407493,52600.563669,42769.569467,62210.455395
2021-01-01,59577.363832,45420.231733,46851.097420,51191.783183,69856.550498,65258.181884,73213.686226,88463.248437,53431.990781,57232.812826,...,51690.816873,54937.257855,51756.144919,58459.528368,60644.388593,55541.542286,67313.293350,54785.733080,44523.785760,63987.006977


# Load Easterlin's data

In [7]:
# Import Easterlin's income data
easterlin_data = pd.read_csv('../historic_data/Historical Statistics of the US - Easterlin State Income Data.csv',index_col=0)

# Import historic CPI data
historic_cpi_data=pd.read_csv('../historic_data/Historical Statistics of the US - cpi.csv',index_col=0)
historic_cpi_data = historic_cpi_data/historic_cpi_data.loc[1929]*float(data_p.loc['1929'])

/var/folders/36/tyk09l294j7b2stxnfxym2200000gp/T/ipykernel_5586/2211716589.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  historic_cpi_data = historic_cpi_data/historic_cpi_data.loc[1929]*float(data_p.loc['1929'])


In [8]:
# Construct series for real incomes in 1840, 1880, and 1900
df_1840 = easterlin_data['Income per capita - 1840 - A [cur dollars]']/float(historic_cpi_data.loc[1840])
df_1880 = easterlin_data['Income per capita - 1880 [cur dollars]']/float(historic_cpi_data.loc[1890])
df_1900 = easterlin_data['Income per capita - 1900 [cur dollars]']/float(historic_cpi_data.loc[1900])

# Put into a DataFrame and concatenate with previous data beginning in 1929
df = pd.DataFrame({pd.to_datetime('1840'):df_1840,pd.to_datetime('1880'):df_1880,pd.to_datetime('1900'):df_1900}).transpose()
df = pd.concat([data_y,df]).sort_index()

/var/folders/36/tyk09l294j7b2stxnfxym2200000gp/T/ipykernel_5586/1796432502.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df_1840 = easterlin_data['Income per capita - 1840 - A [cur dollars]']/float(historic_cpi_data.loc[1840])
/var/folders/36/tyk09l294j7b2stxnfxym2200000gp/T/ipykernel_5586/1796432502.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df_1880 = easterlin_data['Income per capita - 1880 [cur dollars]']/float(historic_cpi_data.loc[1890])
/var/folders/36/tyk09l294j7b2stxnfxym2200000gp/T/ipykernel_5586/1796432502.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  df_1900 = easterlin_data['Income per capita - 1900 [cur dollars]']/float(historic_cpi_data.loc[1900])


In [9]:
# Export data to csv
series = df.sort_index()
dropCols = [u'AK', u'HI', u'New England', u'Mideast', u'Great Lakes', u'Plains', u'Southeast', u'Southwest', u'Rocky Mountain', u'Far West']
for c in dropCols:
    series = series.drop([c],axis=1)

series.to_csv('../csv/state_income_data.csv',na_rep='NaN')